# Unit 12. Classification Models 課堂作業
- 課程編號: CHE-AI-114
- 學年度: 114下
- 上課時間: 每週四 09:10-12:00
- 授課地點: 學思樓 705
- 指導教授: 莊曜禎 助理教授
- 學生姓名: ＯＯＯ
- 學號: m12345678
- email帳號: fcu.m12345678@gmail.com

---
### 0. 環境設定

In [ ]:
from pathlib import Path
import os

# ========================================
# 路徑設定 (兼容 Colab 與 Local)
# ========================================
UNIT_OUTPUT_DIR = 'P3_Unit12_HW'
SOURCE_DATA_DIR = 'winequality'

try:
  from google.colab import drive
  IN_COLAB = True
  print("✓ 偵測到 Colab 環境，準備掛載 Google Drive...")
  drive.mount('/content/drive', force_remount=True)
except ImportError:
  IN_COLAB = False
  print("✓ 偵測到 Local 環境")
try:
  shortcut_path = '/content/CHE-AI-COURSE'
  os.remove(shortcut_path)
except FileNotFoundError:
  pass

if IN_COLAB:
  source_path = Path('/content/drive/My Drive/Colab Notebooks/CHE-AI-COURSE')
  os.symlink(source_path, shortcut_path)
  shortcut_path = Path(shortcut_path)
  if source_path.exists():
    NOTEBOOK_DIR = shortcut_path / 'Part_3' / 'Unit12'
    OUTPUT_DIR = NOTEBOOK_DIR / 'outputs' / UNIT_OUTPUT_DIR
    DATA_DIR = NOTEBOOK_DIR / 'data' / SOURCE_DATA_DIR
    MODEL_DIR = OUTPUT_DIR / 'models'
    FIG_DIR = OUTPUT_DIR / 'figs'
  else:
    print(f"⚠️ 找不到路徑雲端CHE-AI-COURSE路徑，請確認自己的雲端資料夾是否正確")
  
else:
  NOTEBOOK_DIR = Path.cwd()
  OUTPUT_DIR = NOTEBOOK_DIR / 'outputs' / UNIT_OUTPUT_DIR
  DATA_DIR = NOTEBOOK_DIR / 'data' / SOURCE_DATA_DIR
  MODEL_DIR = OUTPUT_DIR / 'models'
  FIG_DIR = OUTPUT_DIR / 'figs'

NOTEBOOK_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
DATA_DIR.mkdir(parents=True, exist_ok=True)
MODEL_DIR.mkdir(parents=True, exist_ok=True)
FIG_DIR.mkdir(parents=True, exist_ok=True)

print(f"\n✓ Notebook工作目錄: {NOTEBOOK_DIR}")
print(f"✓ 數據來源目錄: {DATA_DIR}")
print(f"✓ 結果輸出目錄: {OUTPUT_DIR}")
print(f"✓ 模型輸出目錄: {MODEL_DIR}")
print(f"✓ 圖檔輸出目錄: {FIG_DIR}")

---
### 數據下載

In [ ]:
# 數據來源: Wine Quality Dataset
# https://www.kaggle.com/datasets/yasserh/wine-quality-dataset/

import requests
import zipfile

# 設定路徑與 URL
url = "https://www.kaggle.com/api/v1/datasets/download/yasserh/wine-quality-dataset"
zip_path = DATA_DIR / "wine-quality-dataset.zip"
data_file = DATA_DIR / "WineQT.csv"

def download_and_extract():
    # --- 步驟 A: 下載檔案 ---
    print(f"正在從 {url} 下載...")
    try:
        # allow_redirects=True 處理 Kaggle 的重導向
        response = requests.get(url, allow_redirects=True, stream=True)
        response.raise_for_status() # 若下載失敗會拋出異常
        
        with open(zip_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print(f"下載成功，檔案儲存於: {zip_path}")

        # --- 步驟 B: 解壓縮檔案 ---
        if zipfile.is_zipfile(zip_path):
            print(f"正在解壓縮至: {DATA_DIR}...")
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(DATA_DIR)
            print("解壓縮完成！")
        else:
            print("錯誤：下載的檔案不是有效的 ZIP 格式。")
            
    except Exception as e:
        print(f"發生錯誤: {e}")
        print("\n⚠️ Kaggle API 下載需要認證，請手動下載數據集：")
        print("1. 前往: https://www.kaggle.com/datasets/yasserh/wine-quality-dataset/")
        print(f"2. 下載 WineQT.csv 並放置於: {DATA_DIR}")

if __name__ == "__main__" and not data_file.exists():
    download_and_extract()
else:
    print(f"✓ 檔案已存在於: {data_file}")

---
### 載入相關套件

In [ ]:
# 基礎套件
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# sklearn 套件
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, 
    roc_auc_score, roc_curve
)

# 分類模型
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

# 設定隨機種子以確保結果可重現
SEED = 42
np.random.seed(SEED)

# 設定 matplotlib 中文顯示
plt.rcParams['axes.unicode_minus'] = False

print("✓ 套件載入完成")

---
## I. 練習題: 紅酒品質分類預測

### 數據集說明
本作業使用 **Wine Quality Dataset**，包含紅酒的物理化學特性與品質評分。

**特徵說明** (共 11 個):
- `fixed acidity`: 固定酸度
- `volatile acidity`: 揮發性酸度
- `citric acid`: 檸檬酸含量
- `residual sugar`: 殘糖量
- `chlorides`: 氯化物含量
- `free sulfur dioxide`: 游離二氧化硫
- `total sulfur dioxide`: 總二氧化硫
- `density`: 密度
- `pH`: 酸鹼值
- `sulphates`: 硫酸鹽含量
- `alcohol`: 酒精濃度

**目標變數**:
- `quality`: 品質評分 (3-9分，本作業將轉換為二元分類)
  - **高品質 (1)**: quality >= 7
  - **低品質 (0)**: quality < 7

### 作業目標
使用本單元學習的 6 種分類模型，建立紅酒品質預測模型，並進行綜合比較分析。

---
### 1. 數據準備與探索性分析 (EDA)

**要求**:
1. 載入數據並檢查基本資訊
2. 檢查缺失值與異常值
3. 將 `quality` 轉換為二元分類 (0: 低品質, 1: 高品質)
4. 計算類別分布，檢查是否有類別不平衡問題
5. 繪製至少 3 個特徵的分布圖 (直方圖或箱型圖)
6. 繪製相關性熱力圖 (correlation heatmap)
7. 選擇至少 2 個重要特徵繪製散點圖，並用顏色區分類別

In [ ]:
# 請在此撰寫程式碼


---
### 2. 數據預處理

**要求**:
1. 分離特徵 (X) 與目標變數 (y)
2. 數據分割: 訓練集 70%、測試集 30% (`random_state=42`)
3. 使用 `StandardScaler` 對特徵進行標準化
   - **注意**: 僅對 X 進行標準化，y 不需標準化
   - 先 fit 訓練集，再分別 transform 訓練集和測試集
4. 顯示標準化前後的數據範圍變化

In [ ]:
# 請在此撰寫程式碼


---
### 3. 建立所有分類模型

**要求**: 建立並訓練以下 6 種分類模型

#### 3.1 Logistic Regression (邏輯迴歸)
- 使用預設參數
- 或自行調整 `C`, `penalty`, `solver` 等參數

In [ ]:
# 請在此撰寫程式碼


#### 3.2 Decision Tree Classifier (決策樹分類)
- 設定 `max_depth`, `min_samples_split` 等參數
- 或使用預設參數

In [ ]:
# 請在此撰寫程式碼


#### 3.3 Random Forest Classifier (隨機森林分類)
- 設定 `n_estimators`, `max_depth`, `min_samples_split` 等參數
- 或使用預設參數

In [ ]:
# 請在此撰寫程式碼


#### 3.4 Gradient Boosting Classifier (梯度提升分類)
- 設定 `n_estimators`, `learning_rate`, `max_depth` 等參數
- 或使用預設參數

In [ ]:
# 請在此撰寫程式碼


#### 3.5 Support Vector Classification (支持向量分類)
- 設定 `C`, `kernel`, `gamma` 等參數
- 建議使用 `kernel='rbf'` 或 `'linear'`

In [ ]:
# 請在此撰寫程式碼


#### 3.6 Gaussian Naive Bayes (高斯貝氏分類器)
- 使用預設參數即可

In [ ]:
# 請在此撰寫程式碼


---
### 4. 模型評估與比較

**要求**:
1. 對所有 6 個模型計算以下指標:
   - Accuracy (準確率)
   - Precision (精確率)
   - Recall (召回率)
   - F1-Score
   - ROC-AUC
   
2. 製作比較表格，顯示所有模型的性能指標

3. 繪製比較圖:
   - 6 個模型的 Accuracy 比較 (長條圖)
   - 6 個模型的 F1-Score 比較 (長條圖)
   
4. 選擇表現最佳的模型，繪製:
   - Confusion Matrix (混淆矩陣)
   - ROC Curve (ROC 曲線)
   - Classification Report (分類報告)

5. **分析與討論**:
   - 哪個模型表現最好？為什麼？
   - 哪個模型表現最差？可能的原因？
   - 是否有類別不平衡問題？如何影響模型性能？
   - 在實際應用中，你會選擇哪個模型？理由是什麼？

In [ ]:
# 請在此撰寫程式碼


---
### 5. 模型保存

**要求**:
- 保存表現最佳的模型 (使用 `joblib` 或 `pickle`)
- 保存 StandardScaler (用於未來預測時的標準化)
- 顯示保存的檔案路徑

In [ ]:
# 請在此撰寫程式碼


---
## II. 額外加分作業 (自由繳交)

- 學生可自由選擇是否繳交加分作業 (下週上課前完成 email 電子檔)
- 分數會加在最後總成績上，每份作業加 0.1 ~ 1.0 分
- 分數加的不多，真的不一定要交加分作業，正常出席上課做好期末報告即可
- 加分作業不一定要全部都做完，想繳交至少完成其中一項實驗即可
- 請務必自行完成！你可以問 AI、問同學、問學長姊，甚至參考以前別人的作業，但一定要自行 [吸收][執行][整理] 結果！
- 不要貼 AI 的答案寄給老師看，你自己看就好
- 如果系統自動比對發現作業內容 (與他人雷同、原創性低、抄襲比例過高)，作業的分數會 0 分
- 如果你直接 100% 複製別人的作業繳交，你會直接被當掉 (提供作業給別人複製的也一樣)
- 老師看你作業要花時間，真的有心想寫加分作業，請好好自己做

---
### 實驗1: 超參數調整 (Hyperparameter Tuning)

選擇 **至少 2 個模型** 進行超參數調整

**要求**:
1. 使用 `GridSearchCV` 或 `RandomizedSearchCV`
2. 定義參數網格 (parameter grid)
3. 使用交叉驗證 (至少 3-fold CV)
4. 記錄:
   - 最佳參數組合
   - 調整前後的性能比較
   - 訓練時間
5. **分析**: 超參數調整帶來多少改善？是否值得額外的計算成本？

**建議調整的參數**:
- **Random Forest**: `n_estimators`, `max_depth`, `min_samples_split`, `min_samples_leaf`
- **Gradient Boosting**: `n_estimators`, `learning_rate`, `max_depth`, `subsample`
- **SVC**: `C`, `gamma`, `kernel`
- **Logistic Regression**: `C`, `penalty`, `solver`

In [ ]:
# 請在此撰寫程式碼


---
### 實驗2: 處理類別不平衡 (Class Imbalance)

如果數據集存在類別不平衡問題，嘗試以下方法

**要求**: 至少嘗試 2 種方法

**方法1: SMOTE (合成少數過採樣)**
```python
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)
```

**方法2: 類別權重 (Class Weight)**
```python
# 在模型中設定 class_weight='balanced'
model = RandomForestClassifier(class_weight='balanced', random_state=42)
```

**方法3: 欠採樣 (Under-sampling)**
```python
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=42)
X_train_resampled, y_train_resampled = rus.fit_resample(X_train_scaled, y_train)
```

**分析**:
1. 比較各方法對模型性能的影響
2. 重點觀察 Recall 和 F1-Score 的變化
3. 哪種方法最適合這個數據集？

In [ ]:
# 請在此撰寫程式碼


---
### 實驗3: 特徵工程 (Feature Engineering)

嘗試創建新特徵或選擇重要特徵來改善模型性能

**方法1: 特徵重要性分析**
1. 使用 Random Forest 或 Gradient Boosting 的 `feature_importances_`
2. 繪製特徵重要性圖
3. 選擇 top-k 重要特徵重新訓練模型
4. 比較使用全部特徵 vs. 選擇特徵的性能差異

**方法2: 多項式特徵**
```python
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_scaled)
X_test_poly = poly.transform(X_test_scaled)
```

**方法3: 特徵交互作用**
- 創建有意義的特徵組合 (例如: `alcohol * pH`, `citric_acid / fixed_acidity`)
- 測試這些新特徵是否改善模型性能

**分析**:
- 哪些特徵最重要？
- 特徵工程帶來多少改善？
- 新特徵是否過擬合？

In [ ]:
# 請在此撰寫程式碼


---
### 實驗4: 集成學習 (Ensemble Learning)

結合多個模型來改善預測性能

**方法1: Voting Classifier (投票分類器)**
```python
from sklearn.ensemble import VotingClassifier

# 硬投票 (hard voting)
voting_clf = VotingClassifier(
    estimators=[
        ('lr', LogisticRegression(random_state=42)),
        ('rf', RandomForestClassifier(random_state=42)),
        ('gb', GradientBoostingClassifier(random_state=42))
    ],
    voting='hard'
)

# 軟投票 (soft voting) - 使用機率
voting_clf_soft = VotingClassifier(
    estimators=[...],
    voting='soft'
)
```

**方法2: Stacking Classifier (堆疊分類器)**
```python
from sklearn.ensemble import StackingClassifier

stacking_clf = StackingClassifier(
    estimators=[
        ('rf', RandomForestClassifier(random_state=42)),
        ('gb', GradientBoostingClassifier(random_state=42)),
        ('svc', SVC(probability=True, random_state=42))
    ],
    final_estimator=LogisticRegression(),
    cv=5
)
```

**要求**:
1. 至少實現 1 種集成方法
2. 比較集成模型與單一模型的性能
3. **分析**: 集成學習是否帶來改善？為什麼？

In [ ]:
# 請在此撰寫程式碼


---
### 💭 思考題

請在完成作業後，思考並回答以下問題：

1. **為什麼分類問題的 y 不需要標準化，但 X 需要？**

2. **Accuracy 和 F1-Score 有什麼差別？什麼情況下應該優先考慮 F1-Score？**

3. **如果你是酒廠的品質控制經理，你會選擇 Precision 高的模型還是 Recall 高的模型？為什麼？**

4. **為什麼 Random Forest 通常比 Decision Tree 表現更好？**

5. **SVM 的訓練時間比其他模型長很多，在實際應用中有什麼限制？**

6. **如果測試集的 Accuracy 遠低於訓練集，可能是什麼原因？如何改善？**

7. **Logistic Regression 是線性模型，但為什麼可以用於分類問題？**

8. **在這個紅酒品質預測案例中，False Positive 和 False Negative 哪個更嚴重？為什麼？**

9. **如果未來要預測白酒品質，可以直接使用這個模型嗎？為什麼？**

10. **除了調整超參數，還有哪些方法可以改善模型性能？**

**請在此回答思考題**


---
## 想對老師說的話